### Imports utilizados

In [48]:
import requests
import time
import random
import re
import pandas as pd
import os
from tqdm import tqdm as tqdm
from bs4 import BeautifulSoup

### Parsing da Página Principal da Wikipedia

Cria o diretório para armazenar os verbetes, percorre todo o conjunto e salva o HTML

In [ ]:


def get_infobox(url):
    e = requests.get(url)
    soup = BeautifulSoup(e.text, 'html.parser') 
    tables = soup.findAll('table')
    infobox = ""
    for table in tables:
        if "infobox" in table["class"] or "infobox_v2" in table["class"]:
            infobox = table
            break

    chava = {}
#dando erro. Provavelmente por causa da forma como o título está sendo pego - lavie
    titulo = infobox.findAll("th")[0]
    for tr in soup.findAll('tr'):
        if len(tr.findAll("td")) == 2:
            messi = tr.findAll("td")
            chava[messi[0].text.strip()] = [li.text.strip() for li in messi[1].findAll("li")] if messi[1].findAll("li") else messi[1].text.strip()
            
    print(chava)
    chava["Título"] = titulo.text.strip()
    chava_df = pd.DataFrame.from_dict(chava, orient='index')
    chava_df.transpose().to_json(f'verbetes/{titulo}/{titulo}.json', orient='index')
    print(chava_df.transpose())

        

In [50]:
def cria_pasta(all_verbets, a):
    os.makedirs('verbetes', exist_ok=True)
    for verbet in tqdm(all_verbets):
        titulo = verbet.split('/')[-1]
        soup = BeautifulSoup(a.text, 'html.parser')
        os.makedirs(f'verbetes/{titulo}', exist_ok=True)
        with open(f'verbetes/{titulo}/{titulo}.html', 'w') as f:
            f.write(soup.prettify())
        time.sleep(random.randint(0,1))

In [51]:
a = requests.get('https://pt.wikipedia.org/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal')
soup = BeautifulSoup(a.text, 'html.parser')

Para cada link na seção "main" da página principal, verifica se o link representa um verbete, caso positivo, adiciona à um Set para evitar duplicatas

In [52]:
links = soup.main.find_all('a')
all_verbets = set()
for link in links:
    if link.has_attr('href') and re.match(r'/wiki/.*', link['href']) and not re.match(r'/wiki/.*:.*', link['href']):
            all_verbets.add(link['href'])

Escolhe aleatoriamente um verbet da conjunto obtido da página principal, e faz o mesmo processamento. O loop é executado até que o conjunto de verbetes tenha 5000 verbetes.

In [54]:
while(len(all_verbets) < 115):
    verbet = random.choice(list(all_verbets))
    a = requests.get('https://pt.wikipedia.org' + verbet)
    soup = BeautifulSoup(a.text, 'html.parser')
    links = soup.find_all('a')
    for link in links:
        if link.has_attr('href') and re.match(r'/wiki/.*', link['href']) and not re.match(r'/wiki/.*:.*', link['href']):
            all_verbets.add(link['href'])
            cria_pasta(all_verbets, a)
            get_infobox('https://pt.wikipedia.org' + verbet)
            
    time.sleep(random.randint(1,2))

100%|██████████| 114/114 [00:55<00:00,  2.06it/s]


IndexError: list index out of range